In [ ]:
#Import necessary libraries
import pandas as pd
import matplotlib.pyplot as plt
from google.cloud import storage
import pyarrow.parquet as pq

#Authenticate User
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

Authenticated


In [ ]:
# Initialize a GCS client
storage_client = storage.Client()

# Define bucket and file paths
landing_folder_name = 'landing'
input_file_name = 'nypd_mv_collisions'
output_folder_name = 'cleaned'
output_file_name = 'parquet nypd_mv_collisions'

# Read data from the landing folder file
landing_bucket = storage_client.bucket('my-project-bucket-bc')
landing_blob = landing_bucket.blob(f'{landing_folder_name}/{input_file_name}')
local_file_path = '/tmp/input_file.csv'
landing_blob.download_to_filename(local_file_path)

In [ ]:
# Load data into a DataFrame
df = pd.read_csv(local_file_path)

In [ ]:
#Removed Columns that isn't going to be used in the Project
columns_to_remove= ['latitude','longitude','location','off_street_name','on_street_name','zip_code']
df = df.drop(labels=columns_to_remove, axis=1)

In [ ]:
#Created a filter to display rows with boro filled in
df_boro_filtered = df[df['borough'].notna()]

# Display the DataFrame with rows where 'borough' is not NaN
df_boro_filtered.head()

,borough,contributing_factor_vehicle_1,contributing_factor_vehicle_2,contributing_factor_vehicle_3,contributing_factor_vehicle_4,contributing_factor_vehicle_5,cross_street_name,timestamp,number_of_cyclist_injured,number_of_cyclist_killed,...,number_of_pedestrians_injured,number_of_pedestrians_killed,number_of_persons_injured,number_of_persons_killed,unique_key,vehicle_type_code1,vehicle_type_code2,vehicle_type_code_3,vehicle_type_code_4,vehicle_type_code_5
10774,BROOKLYN,Unspecified,Unspecified,NaN,NaN,NaN,NaN,2016-06-03 15:00:00,0,0,...,0,0,0,0,3456093,4 dr sedan,NaN,NaN,NaN,NaN
20975,BROOKLYN,Traffic Control Disregarded,Unspecified,NaN,NaN,NaN,CADMAN PLAZA EAST,2017-10-28 10:00:00,0,0,...,0,0,1,0,3782416,Station Wagon/Sport Utility Vehicle,Lunch Wagon,NaN,NaN,NaN
22559,QUEENS,Unspecified,NaN,NaN,NaN,NaN,NaN,2019-10-12 20:55:00,1,0,...,0,0,1,0,4228028,E-Sco,NaN,NaN,NaN,NaN
24065,QUEENS,Driver Inattention/Distraction,Unspecified,NaN,NaN,NaN,GRAND CENTRAL PARKWAY,2019-09-26 16:00:00,0,0,...,0,0,0,0,4213652,Sedan,Sedan,NaN,NaN,NaN
24066,QUEENS,Driver Inattention/Distraction,Driver Inattention/Distraction,NaN,NaN,NaN,UTOPIA PARKWAY,2019-04-01 17:00:00,0,0,...,0,0,0,0,4107049,Station Wagon/Sport Utility Vehicle,Sedan,NaN,NaN,NaN


In [ ]:
# Further filter the DataFrame to remove rows where 'contributing_factor_vehicle_1' is NaN
df_CFV1_filtered = df_boro_filtered[df_boro_filtered['contributing_factor_vehicle_1'].notna()]

# Display the final filtered DataFrame
df_CFV1_filtered.head()

,borough,contributing_factor_vehicle_1,contributing_factor_vehicle_2,contributing_factor_vehicle_3,contributing_factor_vehicle_4,contributing_factor_vehicle_5,cross_street_name,timestamp,number_of_cyclist_injured,number_of_cyclist_killed,...,number_of_pedestrians_injured,number_of_pedestrians_killed,number_of_persons_injured,number_of_persons_killed,unique_key,vehicle_type_code1,vehicle_type_code2,vehicle_type_code_3,vehicle_type_code_4,vehicle_type_code_5
10774,BROOKLYN,Unspecified,Unspecified,NaN,NaN,NaN,NaN,2016-06-03 15:00:00,0,0,...,0,0,0,0,3456093,4 dr sedan,NaN,NaN,NaN,NaN
20975,BROOKLYN,Traffic Control Disregarded,Unspecified,NaN,NaN,NaN,CADMAN PLAZA EAST,2017-10-28 10:00:00,0,0,...,0,0,1,0,3782416,Station Wagon/Sport Utility Vehicle,Lunch Wagon,NaN,NaN,NaN
22559,QUEENS,Unspecified,NaN,NaN,NaN,NaN,NaN,2019-10-12 20:55:00,1,0,...,0,0,1,0,4228028,E-Sco,NaN,NaN,NaN,NaN
24065,QUEENS,Driver Inattention/Distraction,Unspecified,NaN,NaN,NaN,GRAND CENTRAL PARKWAY,2019-09-26 16:00:00,0,0,...,0,0,0,0,4213652,Sedan,Sedan,NaN,NaN,NaN
24066,QUEENS,Driver Inattention/Distraction,Driver Inattention/Distraction,NaN,NaN,NaN,UTOPIA PARKWAY,2019-04-01 17:00:00,0,0,...,0,0,0,0,4107049,Station Wagon/Sport Utility Vehicle,Sedan,NaN,NaN,NaN


In [ ]:
#Display incidents where 5 vehicles were involved with no NaN data

# create filter columns
columns_to_filter = df.columns

# Filtered DataFrame with the condition
df_filtered = df[df[columns_to_filter[0]].notna()]

# Iterate over the remaining columns and apply the filtering condition
for column in columns_to_filter[1:]:
    df_filtered = df_filtered[df_filtered[column].notna()]

# Display final filtered DataFrame
df_filtered

,borough,contributing_factor_vehicle_1,contributing_factor_vehicle_2,contributing_factor_vehicle_3,contributing_factor_vehicle_4,contributing_factor_vehicle_5,cross_street_name,timestamp,number_of_cyclist_injured,number_of_cyclist_killed,...,number_of_pedestrians_injured,number_of_pedestrians_killed,number_of_persons_injured,number_of_persons_killed,unique_key,vehicle_type_code1,vehicle_type_code2,vehicle_type_code_3,vehicle_type_code_4,vehicle_type_code_5
37916,MANHATTAN,Fatigued/Drowsy,Unspecified,Unspecified,Unspecified,Unspecified,12 AVENUE,2016-11-07 21:00:00,0,0,...,0,0,2,0,3556442,Box Truck,Sedan,Sedan,Taxi,Sedan
40870,MANHATTAN,Unspecified,Unspecified,Unspecified,Unspecified,Unspecified,MURRAY STREET,2014-09-02 23:57:00,0,0,...,0,0,2,0,1017713,TAXI,PASSENGER VEHICLE,PASSENGER VEHICLE,SPORT UTILITY / STATION WAGON,PASSENGER VEHICLE
41145,MANHATTAN,Unspecified,Unspecified,Unspecified,Unspecified,Unspecified,EAST 9 STREET,2015-11-05 04:15:00,0,0,...,0,0,0,0,3329249,TAXI,SPORT UTILITY / STATION WAGON,SPORT UTILITY / STATION WAGON,PASSENGER VEHICLE,SPORT UTILITY / STATION WAGON
42378,MANHATTAN,Other Vehicular,Traffic Control Disregarded,Driver Inattention/Distraction,Driver Inattention/Distraction,Driver Inattention/Distraction,BOWERY,2016-04-01 22:30:00,0,0,...,0,0,0,0,3406354,4 dr sedan,4 dr sedan,4 dr sedan,4 dr sedan,4 dr sedan
43546,MANHATTAN,Fatigued/Drowsy,Unspecified,Unspecified,Unspecified,Unspecified,HUBERT STREET,2015-07-07 15:54:00,0,0,...,0,0,2,0,3253274,BUS,SPORT UTILITY / STATION WAGON,TAXI,TAXI,SPORT UTILITY / STATION WAGON
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2066491,QUEENS,Headlights Defective,Traffic Control Disregarded,Unspecified,Unspecified,Unspecified,153 STREET,2021-03-26 01:43:00,0,0,...,0,0,1,0,4401898,Van,Sedan,Sedan,Sedan,Sedan
2066725,QUEENS,Unsafe Speed,Unspecified,Unspecified,Unspecified,Unspecified,FOCH BOULEVARD,2018-05-27 21:02:00,0,0,...,0,0,0,0,3909865,Station Wagon/Sport Utility Vehicle,Sedan,Station Wagon/Sport Utility Vehicle,Sedan,Sedan
2066846,QUEENS,Driver Inattention/Distraction,Unspecified,Unspecified,Unspecified,Unspecified,INWOOD STREET,2023-07-29 05:30:00,0,0,...,0,0,0,0,4649597,Sedan,Station Wagon/Sport Utility Vehicle,Sedan,Station Wagon/Sport Utility Vehicle,Station Wagon/Sport Utility Vehicle
2067079,QUEENS,Driver Inattention/Distraction,Unspecified,Unspecified,Unspecified,Unspecified,95 AVENUE,2021-12-01 15:00:00,0,0,...,0,0,0,0,4484727,Sedan,Sedan,Sedan,Sedan,Sedan


In [ ]:
missing_counts = df_filtered.isna().sum()
missing_counts

borough                          0
contributing_factor_vehicle_1    0
contributing_factor_vehicle_2    0
contributing_factor_vehicle_3    0
contributing_factor_vehicle_4    0
contributing_factor_vehicle_5    0
cross_street_name                0
timestamp                        0
number_of_cyclist_injured        0
number_of_cyclist_killed         0
number_of_motorist_injured       0
number_of_motorist_killed        0
number_of_pedestrians_injured    0
number_of_pedestrians_killed     0
number_of_persons_injured        0
number_of_persons_killed         0
unique_key                       0
vehicle_type_code1               0
vehicle_type_code2               0
vehicle_type_code_3              0
vehicle_type_code_4              0
vehicle_type_code_5              0
dtype: int64

In [ ]:
# Save the outputs to a new file in the output folder
output_bucket = storage_client.bucket('my-project-bucket-bc')
output_blob = output_bucket.blob(f'{output_folder_name}/{output_file_name}')

# Save description DataFrame to Parquet format
output_file_path = '/tmp/output_file.parquet'
df_filtered.to_parquet(output_file_path, engine='pyarrow')

# Upload Parquet file to GCS
output_blob.upload_from_filename(output_file_path)

print("Outputs saved successfully as Parquet.")

Outputs saved successfully as Parquet.
